**Description**: Shows how to compare MIP formulations for transportation problems with piecewise linear objective functions.

**Author**: Juan Pablo Vielma

**License**: <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

# MIP formulations for piecewise linear functions  with JuMP

We consider the transportation problem
$$\begin{alignedat}{3}
 &\min & \quad  \sum_{s=1}^{n_s} \sum_{d=1}^{n_d} f_{s,d}(x_{s,d})  \\
 \notag&s.t.\\
 &      &  \sum_{d=1}^{n_d} x_{s,d} &= supply_s,  &\quad& \forall s\in \{1,\ldots,n_s\} \\
  &      &   \sum_{s=1}^{n_s} x_{s,d} &= demand_d,  &\quad& \forall d\in \{1,\ldots,n_d\} \\
  &      &  x_{s,d} &\geq 0,  &\quad& \forall s\in \{1,\ldots,n_s\},\quad d\in \{1,\ldots,n_d\} 
 \end{alignedat}$$
 where $ f_{s,d}:\mathbb{R}\to \mathbb{R}$ is a piecewise linear function. 

We begin constructing a Linear Programming formulation for the case in which all $ f_{s,d}$ are linear and then consider Mixed Integer Programming formulations for the general case. 

In [ ]:
using JuMP, Cbc

In [ ]:
# Create JuMP model, using Cbc as the solver
model = Model(solver=CbcSolver())

# Data
m = n = 3
S = 1:m
D = 1:n
supply = rand(1:10, m)
demand = rand(1:10, n)
demand_imbalance = sum(demand) - sum(supply)
if demand_imbalance > 0
    supply[m] += demand_imbalance
else
    demand[n] -= demand_imbalance
end    
c = rand(m,n)

# Define variables
@variable(model, x[S,D] >= 0)

# Add constraints
@constraints(model, begin    
    supply_constraints[s=S], sum(x[s,d] for d in D) == supply[s]
    demand_constraints[d=D], sum(x[s,d] for s in S) == demand[d]    
end)

# Define objective
@objective(model, Min, sum(c[s,d]*x[s,d] for  s in S, d in D))

# Display model
println(model)

# Solve Model
println("Solving...")
status = solve(model)

# Display results
println("Solver status: ", status)
println("Cost: ", getobjectivevalue(model))
println("Solution: \n",getvalue(x))

In [ ]:
# Generate supply, demand and piecewise linear objective
function generateData(m, n, num_segments)
    
#     seed = MersenneTwister(hash((num_segments,m,n)))
    
    S = 1:m
    D = 1:n
#     supply = rand(seed, 1:10, m)
#     demand = rand(seed, 1:10, n)
    supply = rand(1:10, m)
    demand = rand(1:10, n)
    
    demand_imbalance = sum(demand) - sum(supply)
    if demand_imbalance > 0
        supply[m] += demand_imbalance
    else
        demand[n] -= demand_imbalance
    end
    
    fvalues = [zeros(num_segments+1) for s in S, d in D]
    dvalues = [zeros(num_segments+1) for s in S, d in D]
    for s in S, d in D
        drange = min(supply[s],demand[d])
        Δ = drange / num_segments
        dvalues[s,d] = [(i-1)*Δ for i=1:(num_segments+1)]
            
        slopes = sort(rand(num_segments), rev=true)
        for i in 2:(num_segments+1)
            fvalues[s,d][i] = fvalues[s,d][i-1] + slopes[i-1]*(dvalues[s,d][i]-dvalues[s,d][i-1])
        end
    end
    
    supply, demand, fvalues, dvalues
end

supply, demand, fvalues, dvalues = generateData(1,1,8)
snodes = length(supply); dnodes = length(demand);

In [ ]:
# Plot a piecewise linear objective
using Plots
plot(dvalues[1,1],fvalues[1,1], legend=false)

In [ ]:
using PiecewiseLinearOpt

# Generate Data
m = 1
n = 2
S = 1:m
D = 1:n
supply, demand, fvalues, dvalues = generateData(m,n,3)

# Create JuMP model, using Cbc as the solver
model = Model(solver=CbcSolver())

# Define variables
@variable(model, x[S,D] >= 0)

# Add constraints
@constraints(model, begin    
    supply_constraints[s=S], sum(x[s,d] for d in D) == supply[s]
    demand_constraints[d=D], sum(x[s,d] for s in S) == demand[d]    
end)

# Define objective
@objective(model, Min, sum(piecewiselinear(model, x[s,d], dvalues[s,d], fvalues[s,d]) for s in S, d in D))

# Display Model
println(model)

# Solve Model
println("Solving...")
status = solve(model)

# Display results
println("Solver status: ", status)
println("Cost: ", getobjectivevalue(model))
println("Solution: \n",getvalue(x))

In [ ]:
function solveModel(supply, demand, fvalues, dvalues, formulation, solver=CbcSolver())
    S = 1:length(supply)
    D = 1:length(demand)
    model = Model(solver=solver)

    @variable(model, x[S,D] >= 0)

    # Add constraints
    @constraints(model, begin    
        supply_constraints[s=S], sum(x[s,d] for d in D) == supply[s]
        demand_constraints[d=D], sum(x[s,d] for s in S) == demand[d]    
    end)

    obj = sum(piecewiselinear(model, x[s,d], dvalues[s,d], fvalues[s,d]; method=formulation) for s in S, d in D)
    @objective(model, Min, obj)
    
    println("Solving for ", formulation, " ...")
    tic()
    status = solve(model)
    solvetime = toq();
    println("Solver status: ", status)
    println("Cost: ", getobjectivevalue(model))
    println("Solve time: ", solvetime)
    println()
end

In [ ]:
# Generate Data
supply, demand, fvalues, dvalues = generateData(10, 10, 30)

formulations = [:CC,:MC,:SOS2,:Incremental,:Logarithmic,:DisaggLogarithmic,:ZigZag,:ZigZagInteger]

for formulation in formulations
    solveModel(supply, demand, fvalues, dvalues, formulation, CbcSolver(logLevel=0, seconds=60.))
end

In [ ]:
using Gurobi

for formulation in formulations
    solveModel(supply, demand, fvalues, dvalues, formulation,GurobiSolver(TimeLimit=60,OutputFlag=0))
end